# Train Multi-GPU single machine

Run on V100 8x in tests.

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re, json
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import dask
import time
print(tf.__version__)

2.4.1


In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 4 Logical GPUs


In [4]:
def train_multigpu(n_epochs, classes, base_lr, batchsize,wbargs, scale_batch = False, scale_lr = False):
    
    wbargs={**wbargs, "scale_batch":scale_batch, "scale_lr":scale_lr}

    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='tensorflow_mg_birds',
    config=wbargs)

    strategy = tf.distribute.MirroredStrategy()
    print('Number of devices: %d' % strategy.num_replicas_in_sync)
    if scale_batch:
        batchsize = batchsize*strategy.num_replicas_in_sync
        
    if scale_lr:
        base_lr = base_lr*strategy.num_replicas_in_sync
    
    with strategy.scope():
        model = tf.keras.applications.ResNet50(
            include_top=True,
            weights=None,
            classes=classes)

        optimizer = keras.optimizers.Adam(lr=base_lr)
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #integer labels

    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/tensorflow_ds/datasets/birds/285 birds/train',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2).cache().shuffle(1000)
        
    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/tensorflow_ds/datasets/birds/285 birds/valid',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2)
    
    start = time.time()

    model.fit(
        train_ds, 
        epochs=n_epochs, 
        validation_data=valid_ds,
        callbacks=[WandbCallback()]
    )
    end = time.time()-start
    print("model training time", end)
    wandb.log({"training_time":end})

    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras_multi/')

In [5]:
model_params = {'n_epochs': 10, 
                'base_lr': .02,
               'batchsize': 512,
                   'classes':285}

wbargs = {**model_params,
    'Notes':"tf_v100_8x",
    'Tags': ['multi', 'gpu', 'tensorflow'],
    'dataset':"Clothing",
    'architecture':"ResNet50"}

In [6]:
tester_plain = train_multigpu(wbargs=wbargs, **model_params)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:loc

KeyboardInterrupt: 

In [ ]:
#tester_batchscale = train_multigpu(scale_batch = True, wbargs=wbargs, **model_params)

In [ ]:
# tester_lrscale = train_multigpu(scale_lr = True, wbargs=wbargs, **model_params)

In [ ]:
#tester_both = train_multigpu(scale_lr = True, scale_batch = True, wbargs=wbargs, **model_params)

In [ ]:

# USE categorical_crossentropy

#     # Data
#     tester_flow = image.ImageDataGenerator()
#     train_ds = tester_flow.flow_from_directory(
#         directory = '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
#         target_size=(224,224),
#         batch_size=batch_size_dist
#     )

#     valid_ds = tester_flow.flow_from_directory(
#         directory = '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/validation',
#         target_size=(224,224),
#         batch_size=batch_size_dist
#     )

#     # Data
    # Data
#     train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#         '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
#         image_size=(224,224),
#         batch_size=batchsize
#     ).prefetch(2).cache().shuffle(1000)
        
#     valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
#         '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/validation',
#         image_size=(224,224),
#         batch_size=batchsize
#     ).prefetch(2)